In [1]:
import re

In [4]:
def remove_instance_id(title):
    return re.sub('i\-[^\s]+\s', '', title)

In [27]:
def get_instance_id(title):
    return re.search('i\-[^\s]+', title)

In [23]:
strs = [
    'Instance i-0c8dc35fee332a9c1 is vulnerable to CVE-2019-20388',
    'Instance i-0ac3a959c08a8ea6a is vulnerable to CVE-2019-20388',
    'Instance i-01dd7679aec513da5 is vulnerable to CVE-2019-20388',
    'Instance i-0007a942207995d1e is vulnerable to CVE-2019-17546',
    'Instance i-0ac3a959c08a8ea6a is vulnerable to CVE-2019-17546'
]

[remove_instance_id(s) for s in strs]

['Instance is vulnerable to CVE-2019-20388',
 'Instance is vulnerable to CVE-2019-20388',
 'Instance is vulnerable to CVE-2019-20388',
 'Instance is vulnerable to CVE-2019-17546',
 'Instance is vulnerable to CVE-2019-17546']

In [29]:
[get_instance_id(s).group(0) for s in strs]

['i-0c8dc35fee332a9c1',
 'i-0ac3a959c08a8ea6a',
 'i-01dd7679aec513da5',
 'i-0007a942207995d1e',
 'i-0ac3a959c08a8ea6a']

In [40]:
mydict = {'key1': 'val1', 'key2': 'val2'}

In [46]:
if 'key5' not in mydict:
    mydict['key5'] = {
        'id': ['id1'],
        'name': ['name1']
    }
else:
    print('there')

In [47]:
mydict

{'key1': 'val1',
 'key2': 'val2',
 'key4': 'id1',
 'key5': {'id': ['id1'], 'name': ['name1']}}

In [37]:
len(mydict.keys())

3

In [49]:
mydict['key5']['id'].append('id2')

In [50]:
mydict

{'key1': 'val1',
 'key2': 'val2',
 'key4': 'id1',
 'key5': {'id': ['id1', 'id2'], 'name': ['name1']}}

In [51]:
import boto3
import json
import re
import datetime
import os

clientInspector = boto3.client('inspector')
clientS3 = boto3.client('s3')
sns = boto3.client('sns')

def lambda_handler(event, context):
    records = event["Records"]
    for v in records:
        message = (v["Sns"]["Message"])
        create_dict = json.loads(message)
        run_arn = create_dict.get('run')
        
    important_findings, ignore_count = obtain_findings(run_arn=run_arn)
    
    # Send to S3
    send_findings_to_s3(run_arn, important_findings)

    # Notify Pagerduty through SNS email
    email_report(run_arn, len(important_findings.keys()), ignore_count)
    

def obtain_findings(run_arn):
    # Get the unique findings of the Inspector run and consolidate the affected EC2 instance IDs and AMIs
    try:
        paginator = clientInspector.get_paginator('list_findings')
        response_iterator = paginator.paginate(assessmentRunArns=[run_arn])
        response_describe_finding = None
        ignore_attribute = {
            'key': 'action',
            'value': 'ignore'
        }
        important_findings = {}
        ignore_count = 0
        
        for response in response_iterator:
            for k,v in response.items():
                if k == "findingArns":
                    if v == []:
                        continue
                    else:
                        list_find = (v)
                        for idx, arn in enumerate(list_find):
                            response_describe_finding = clientInspector.describe_findings(findingArns=[arn],locale='EN_US')
                            finding_title = ''
                            
                            if len(response_describe_finding['findings']) > 0:
                                finding_title = response_describe_finding['findings'][0]['title']
                            
                            title_no_instance = remove_instance_id(finding_title)
                            instance_id = response_describe_finding['findings'][0]['assetAttributes']['agentId']
                            ami_id = response_describe_finding['findings'][0]['assetAttributes']['amiId']
                            
                            # important_findings stores only the unique findings, and each finding contains a list of affected EC2 instances
                            if title_no_instance not in important_findings:
                                important_findings[title_no_instance] = {
                                    'severity': response_describe_finding['findings'][0]['severity'],
                                    'instances': [instance_id],
                                    'ami': [ami_id]
                                }
                            else:
                                # Add instance id and ami id to the list of instances affected by the finding
                                important_findings[title_no_instance]['instances'].append(instance_id)
                                
                                if ami_id not in important_findings[title_no_instance]['ami']:
                                    important_findings[title_no_instance]['ami'].append(ami_id)
                                
                                # Put an ignore tag on similar findings on other EC2 instances
                                clientInspector.add_attributes_to_findings(findingArns=[arn], attributes=[ignore_attribute])
                                ignore_count += 1
                                
                                print('Skipped finding arn: ' + arn)
                                
                                continue
                
        return important_findings, ignore_count
                
    except Exception as error:
        print(error)
    
    
def remove_instance_id(title):
    return re.sub('i\-[^\s]+\s', '', title)

def send_findings_to_s3(run_arn, important_findings):
    print('Sending important findings to S3')

    try:
        filename = 'findings_' + str(datetime.datetime.now()) + '.json'
        json_upload = bytes(json.dumps(
                        {'run_arn': run_arn,
                        'findings': important_findings},
                    indent=4).encode('UTF-8'))
            
        clientS3.put_object(Bucket=os.environ['BUCKET'], Key=filename, Body=json_upload)
        print('Upload to S3 complete')

    except Exception as error:
        print('Error uploading findings to S3')
        print(error)

def email_report(run_arn, finding_count, ignore_count):
    subject = "AWS Inspector Run Findings Report - " + str(datetime.datetime.now())
    messageBody = "Inspector has finished assessment with 0 significant findings.\n"
    
    if finding_count > 0:
        messageBody = "Inspector has finished assessment with {0} significant findings and {1} ignored findings. Go to Inspector console or S3 for details.\n\n".format(str(finding_count), ignore_count)

    print("Sending SNS notification")
    
    try:
        response = sns.publish(
            TopicArn = os.environ['SNS_TOPIC_ARN'],
            Message = messageBody,
            Subject = subject
            )

        print("SNS Notification sent", response)

    except Exception as error:
        print('Failed to deliver message to SNS')
        print(error)
        
    
